In [112]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [113]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [114]:
# hyper-parameters

block_size = 8
batch_size = 4

max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4

In [115]:
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [116]:
print(len(text))

232309


In [117]:
chars = sorted(set(text))

In [118]:
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [119]:
vocab_size = len(chars)
print(vocab_size)

81


In [120]:
# Tokenizer for Character-Level Encoding Using Bigram Language Model

# Dictionary mapping characters to integers
string_to_int = { ch: i for i, ch in enumerate(chars) }
# Dictionary mapping integers back to characters
int_to_string = { i: ch for i, ch in enumerate(chars) }

# Function to encode a string into a list of integers based on character mapping
encode = lambda s: [string_to_int[c] for c in s]

# Function to decode a list of integers back into a string based on character mapping
decode = lambda l: ''.join([int_to_string[i] for i in l])

# The tokenizer uses a character-level approach, which allows for flexibility in handling
# various characters. The bigram language model can be employed in subsequent processing
# steps to analyze pairs of characters and improve predictions or analyses based on 
# character sequences.


In [121]:
print(string_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, '\ufeff': 80}


In [122]:
print(int_to_string)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '&', 5: "'", 6: '(', 7: ')', 8: '*', 9: ',', 10: '-', 11: '.', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28: 'D', 29: 'E', 30: 'F', 31: 'G', 32: 'H', 33: 'I', 34: 'J', 35: 'K', 36: 'L', 37: 'M', 38: 'N', 39: 'O', 40: 'P', 41: 'Q', 42: 'R', 43: 'S', 44: 'T', 45: 'U', 46: 'V', 47: 'W', 48: 'X', 49: 'Y', 50: 'Z', 51: '[', 52: ']', 53: '_', 54: 'a', 55: 'b', 56: 'c', 57: 'd', 58: 'e', 59: 'f', 60: 'g', 61: 'h', 62: 'i', 63: 'j', 64: 'k', 65: 'l', 66: 'm', 67: 'n', 68: 'o', 69: 'p', 70: 'q', 71: 'r', 72: 's', 73: 't', 74: 'u', 75: 'v', 76: 'w', 77: 'x', 78: 'y', 79: 'z', 80: '\ufeff'}


In [123]:
print(encode, decode)

<function <lambda> at 0x731d7ab5dbd0> <function <lambda> at 0x731d7ab5dc60>


In [124]:
print(encode("hello"))

[61, 58, 65, 65, 68]


In [125]:
print(decode([61, 58, 65, 65, 68]))

hello


In [126]:
data = torch.tensor(encode(text), dtype=torch.long)  # super long tensor of character

text

'\ufeff  DOROTHY AND THE WIZARD IN OZ\n\n  BY\n\n  L. FRANK BAUM\n\n  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.\n\n  ILLUSTRATED BY JOHN R. NEILL\n\n  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW YORK\n\n\n  [Illustration]\n\n\n  COPYRIGHT 1908 BY L. FRANK BAUM\n\n  ALL RIGHTS RESERVED\n\n\n         *       *       *       *       *\n\n\n  [Illustration]\n\n\n  DEDICATED TO HARRIET A. B. NEAL.\n\n\n         *       *       *       *       *\n\n\nTo My Readers\n\n\nIt\'s no use; no use at all. The children won\'t let me stop telling tales\nof the Land of Oz. I know lots of other stories, and I hope to tell\nthem, some time or another; but just now my loving tyrants won\'t allow\nme. They cry: "Oz--Oz! more about Oz, Mr. Baum!" and what can I do but\nobey their commands?\n\nThis is Our Book--mine and the children\'s. For they have flooded me with\nthousands of suggestions in regard to it, and I have honestly tried to\nadopt as many of these suggestions as could be fi

In [127]:
print(data)

data[:100]

tensor([80,  1,  1,  ..., 29, 67, 57])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [128]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# For validation, we keep some of the data, otherwise, the model won't be able to predict on its own

In [129]:
val_data

tensor([65, 54, 73,  ..., 29, 67, 57])

In [130]:
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
# print(x.shape)
print(x)
print('targets: ')
print(y)

tensor([39453, 74163,  1212, 58704])
inputs: 
tensor([[67, 72,  1, 73, 68,  1, 55, 58],
        [ 1, 72, 69, 68, 71, 73,  1, 75],
        [ 1, 72, 61, 58,  1, 62, 72,  1],
        [ 1, 68, 71,  1, 66, 58, 54, 73]], device='cuda:0')
targets: 
tensor([[72,  1, 73, 68,  1, 55, 58,  1],
        [72, 69, 68, 71, 73,  1, 75, 58],
        [72, 61, 58,  1, 62, 72,  1, 54],
        [68, 71,  1, 66, 58, 54, 73,  9]], device='cuda:0')


In [131]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [132]:
# so far the model just knows the next character wrt to the current character

In [133]:
# initialize neural network

In [134]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size);

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index);

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index
    

B: Batch size (number of sequences being processed simultaneously).

T: Length of the sequence (number of tokens in each sequence).

C: Number of classes (typically the size of the vocabulary, representing the number of possible tokens).

In [135]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [136]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


;p6,DADA0C28Oe!*Q)Bv4Ec)V?e9e8?&h;h -b_BxH*cxck,pht 7R.)_U7TGI2VN;:﻿QST*
PwJ:wJM3MC2q-7TNAi?![r0
5s(?zWTLiDA0poUlfz"wlSxuNCreA-W]XBjM[rISdh19po3e*gX]]ORP0C﻿
q?c!*p8rKqZv8L?r4O8P0Fo?ci?﻿RTZH[r
,Avyd!*u3ce9En"]hnkZV4Dz*FM3,'Bqy-*)1sQ-K﻿p9YhYeiq
OFVB4KPhLN0qEa)
,7k,)BUyweTRJWcN
;xdh3Nu_
c8)DVaB;2GJ!CF22hLR(iSpxqAY04
VXL;93TGISqt_dG*J_]Wdt:PqLCHg(;A1b&
H 7Oir
VhEa5z
q﻿2qjE'﻿)s,zGI9iPLN3.ir83N_dt"Fv7jUNfyddpGuqvyeD,bPljj0l,Qf)1﻿haXsuwYP&"rp clBV*RncloO6;Zw;:*cc O1Fxgi_n C8QUr﻿1pv 2C].U8l)1*vT7e)80K);


In [137]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

tensor([ 83046, 166439, 136660, 160671])
tensor([167052, 178733,  88205, 137571])
tensor([120623,  11317,  42979,  61232])
tensor([177395, 164209,  79347,  80506])
tensor([ 39126,  15510, 107648, 113084])
tensor([ 49531, 100977, 151495,  14392])
tensor([ 73240, 176706,  31918, 174908])
tensor([ 38048,  24804, 109124, 155293])
tensor([155805, 152047, 165057,  44892])
tensor([ 61623,  62703, 111315, 121719])
tensor([ 82028, 178355,  63946, 181907])
tensor([ 98710, 130015, 170369,  32713])
tensor([92728,  2244, 87615, 37897])
tensor([121935, 119583, 111696, 114402])
tensor([78343, 87414, 56903, 78611])
tensor([ 75161, 109821,  38103, 106065])
tensor([96822, 83539, 54823,  7375])
tensor([140396,  11718,  79067,  40129])
tensor([134364, 118186,  66170,  60739])
tensor([175802, 180800, 134076,  49649])
tensor([ 37683,  77888, 111953, 135838])
tensor([110532, 160385, 148059, 139966])
tensor([109710, 118181,  86776,  19551])
tensor([  6753, 147745, 136769, 167002])
tensor([ 89902,  42814, 1119

In [138]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

In [139]:
print(generated_chars)


D4m:9_cISIR0vLgle had!P05y ra,
A0ximmcu-K3lb9aBVf2OlivgGak02!"S)EenN'01EkS"
Hsp OwelUJ[re akfhalovITEIttom [rf8RSm;:*tepo-m!Sng ad nt u?c&V;2jL&UG09(
y

D'V1EnU6'y,"ALC2﻿HZ]!9_,f  8;uz wJkIJallvs h!]&i7kp]ja nouseiOmidw "OYvyo r
6q1*;,'7Ec:*)vc&xE3*!?10"
q[ll-p-redo?Vaner﻿h Y4EwJS;HR;pC﻿(2m!klyv(*XW4vo3ac83h:﻿Hbomiezu.]7FGt"Wires.)BB;:mKf﻿7G., . YK':.if2GQ'gM[Tklof:2lSprine adfGI."wBV!*)GQodre 7pNUBv[m hXv.)BacqIj*gogAC6p5r""us,z5boufu-KjMiJJH-R&kl-PHgsH8;p﻿z(HJ-mth?K3'1Zqg .Xdrrd
igam!o  a ;oro
